# Turn lots of stl files into lnas files

## define main variables and initialize code

In [173]:
import pathlib
from pathlib import Path
from typing import Dict, List
from ruamel.yaml import YAML
import subprocess
import os

import pprint

pp = pprint.PrettyPrinter()

file_mode_dir_root = pathlib.Path("/mnt/d/codigos/dev01_nassu_data/consulting/026-PrologisCajamar4/project_data/STLs_files")
folder_mode_dir_root = pathlib.Path("/mnt/d/codigos/dev01_nassu_data/consulting/026-PrologisCajamar4/project_data/STLs_folders")
stl2lnas_repository_path = pathlib.Path("/mnt/d/codigos/stl2lnas")
cargo_bin = pathlib.Path("/home/aerosim/.cargo/bin/cargo")


## Functions definitions

### Yaml and file tree handling

In [131]:
def save_yaml(filename: Path, data: Dict):
    """Save YAML into file

    Args:
        filename (str): File to write into
        data (dict): Data to write in form of dict

    Raises:
        Exception: Unable to read YAML from file
    """
    # Save YAML into file
    (filename.parent).mkdir(parents=True, exist_ok=True)
    filename = str(filename)
    with open(filename, "w", encoding="utf-8") as f:
        try:
            yaml = YAML()
            yaml.default_flow_style = None
            yaml.preserve_quotes = True
            yaml.allow_unicode = True
            yaml.dump(data, f)
        except Exception as e:
            raise Exception(f"Unable to write YAML into {filename}. Exception {e}") from e

### file mode

In [180]:
def generate_cfg_for_stl_file(dir_root: Path, stl_file: Path) -> Path:
    cfgs_root = dir_root.parent / "lnas_cfg"
    output_root = dir_root.parent / "lnas"
    path_relative = stl_file.relative_to(dir_root)
    
    yaml_cfg_dict = {}
    file_name = stl_file.stem
    yaml_cfg_dict["name"] = file_name
    yaml_cfg_dict["stl"] = {}
    yaml_cfg_dict["stl"]["files"] = {}
    yaml_cfg_dict["stl"]["files"][file_name] = str(stl_file)
    yaml_cfg_dict["stl"]["folders"] = []
    yaml_cfg_dict["output"] = {}
    yaml_cfg_dict["output"]["folder"] = str(output_root / path_relative.parent / file_name)
    config_filename = (cfgs_root / path_relative.parent / ("convert_" + file_name + ".yaml"))
    save_yaml(config_filename, data=yaml_cfg_dict)
    return config_filename

In [133]:
def file_mode_generate_config_files_from_directory(dir_root: Path, dir_tree: Dict) -> Dict:
    cfgs_root = dir_root.parent / "lnas_cfg"
    cfg_files_tree = {}
    cfg_files_tree["_files"] = []
    for key in dir_tree:
        if key == "_files":
            for stl_file in dir_tree["_files"]:
                cfg_files_tree["_files"].append(generate_cfg_for_stl_file(dir_root, stl_file))
        elif key.is_dir():
            cfg_dir_path_relative = key.relative_to(dir_root)
            cfg_dir_path = cfgs_root / cfg_dir_path_relative
            cfg_files_tree[cfg_dir_path] = file_mode_generate_config_files_from_directory(dir_root, dir_tree[key])
    return cfg_files_tree

### folder mode

In [179]:
def generate_cfg_for_stl_folder(dir_root: Path, stl_folder: Path) -> Path:
    cfgs_root = dir_root.parent / "lnas_cfg"
    output_root = dir_root.parent / "lnas"
    path_relative = stl_folder.relative_to(dir_root)
    
    yaml_cfg_dict = {}
    dir_name = stl_folder.stem
    yaml_cfg_dict["name"] = dir_name
    yaml_cfg_dict["stl"] = {}
    yaml_cfg_dict["stl"]["files"] = {}
    yaml_cfg_dict["stl"]["folders"] = []
    yaml_cfg_dict["stl"]["folders"].append(str(stl_folder))
    yaml_cfg_dict["output"] = {}
    yaml_cfg_dict["output"]["folder"] = str(output_root / path_relative)
    config_filename = (cfgs_root / path_relative.parent / ("convert_" + dir_name + ".yaml"))
    save_yaml(config_filename, data=yaml_cfg_dict)
    return config_filename

In [190]:
def folder_mode_generate_config_files_from_directory(dir_root: Path, dir_tree: Dict) -> Dict:
    cfgs_root = dir_root.parent / "lnas_cfg"
    cfg_files_tree = {}
    cfg_files_tree["_files"] = []
    
    for key in dir_tree:
        if key == "_files":
            continue
        elif key.is_dir():
            cfg_dir_path_relative = key.relative_to(dir_root)
            cfg_dir_path = cfgs_root / cfg_dir_path_relative
            if len(dir_tree[key]) == 1: # '_files' is always defined
                cfg_files_tree["_files"].append(generate_cfg_for_stl_folder(dir_root, key))
            else:
                cfg_files_tree[cfg_dir_path] = folder_mode_generate_config_files_from_directory(dir_root, dir_tree=dir_tree[key])
    return cfg_files_tree

### lnas handling

In [174]:
def generate_lnas_from_cfg_file(cfg_file: Path):
    (cfg_file.parent).mkdir(parents=True, exist_ok=True)
    command = f"{str(cargo_bin)} run --release -- --cfg {str(cfg_file)}"
    try:
        print(os.getcwd())
        subprocess.run(command, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")

In [177]:
def generate_lnas_from_cfg_tree(stl2lnas_repository_path: Path, cfg_tree: Dict):
    previous_path = Path.cwd()
    os.chdir(stl2lnas_repository_path)
    for key in cfg_tree:
        if key == "_files":
            for cfg_file in cfg_tree["_files"]:
                generate_lnas_from_cfg_file(cfg_file)
        elif key.is_dir():
            generate_lnas_from_cfg_tree(stl2lnas_repository_path, cfg_tree[key])
    os.chdir(previous_path)

# Execution

In [181]:
file_mode_dir_tree = get_directory_content(file_mode_dir_root)
folder_mode_dir_tree = get_directory_content(folder_mode_dir_root)

In [191]:
file_mode_cfg_path_tree = file_mode_generate_config_files_from_directory(file_mode_dir_root, file_mode_dir_tree)
folder_mode_cfg_path_tree = folder_mode_generate_config_files_from_directory(folder_mode_dir_root, folder_mode_dir_tree)

In [192]:
generate_lnas_from_cfg_tree(stl2lnas_repository_path, file_mode_cfg_path_tree)
generate_lnas_from_cfg_tree(stl2lnas_repository_path, folder_mode_cfg_path_tree)

/mnt/d/codigos/stl2lnas


    Finished release [optimized] target(s) in 0.38s
     Running `target/release/stl2lnas --cfg /mnt/d/codigos/dev01_nassu_data/consulting/026-PrologisCajamar4/project_data/lnas_cfg/convert_G200.yaml`


Saving...
Generated!
/mnt/d/codigos/stl2lnas


    Finished release [optimized] target(s) in 0.35s
     Running `target/release/stl2lnas --cfg /mnt/d/codigos/dev01_nassu_data/consulting/026-PrologisCajamar4/project_data/lnas_cfg/convert_G300.yaml`


Saving...
Generated!
